In [11]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [23]:
df = pd.read_csv('corrected_data.csv', skipinitialspace=True, sep=',', index_col='id')
df.head()

,год,время_года,климат,город,страна,способ_охлаждения,режим_при_смешанном_типе_охлаждения,способ_обогрева,возраст,пол,...,скорость_воздуха,рост,вес,занавески,вентилятор,окно,двери,отопление,среднемесячная_температура_на_улице,количество_рекламаций
id,,,,,,,,,,,,,,,,,,,,,
0,2011,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,22,Мужской,...,0.26,170.2,63.0,1.0,1.0,NaN,NaN,NaN,28.9,0
1,2011,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,40,Мужской,...,0.12,170.2,63.0,0.0,0.0,NaN,NaN,NaN,84.0,0
2,2011,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,42,Мужской,...,0.08,170.2,63.0,0.0,0.0,NaN,NaN,NaN,32.8,0
3,2011,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,27,Мужской,...,27.16,170.2,63.0,1.0,1.0,NaN,NaN,NaN,28.9,1
4,2011,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,46,Мужской,...,63.83,170.2,63.0,1.0,1.0,NaN,NaN,NaN,32.8,0


In [3]:
df['режим_при_смешанном_типе_охлаждения'] = df['режим_при_смешанном_типе_охлаждения'].fillna('нет')
df['способ_обогрева'] = df['способ_обогрева'].fillna('нет')

In [ ]:
gb = df.groupby(['климат','способ_охлаждения','способ_обогрева'])['оценка_комфорта']
md = gb.median()
for climat, cooling, heating in md.index:
    if not md.isna()[climat, cooling, heating]:
        df['оценка_комфорта'][(df['климат'] == climat) & 
               (df['способ_охлаждения'] == cooling) & 
               (df['способ_обогрева'] == heating)] = df['оценка_комфорта'][(df['климат'] == climat) & 
                                                                (df['способ_охлаждения'] == cooling) & 
                                                                (df['способ_обогрева'] == heating)].fillna(md[climat, cooling, heating])
    else:
        df['оценка_комфорта'][(df['климат'] == climat) & 
               (df['способ_охлаждения'] == cooling) & 
               (df['способ_обогрева'] == heating)] = df['оценка_комфорта'][(df['климат'] == climat) & 
                                                                (df['способ_охлаждения'] == cooling) & 
                                                                (df['способ_обогрева'] == heating)].fillna(df['оценка_комфорта'].median())

In [25]:
gb = df.groupby('город')['возраст']
md = gb.median()
for city in md.index:
    if not md.isna()[city]:
        df['возраст'][df['город'] == city] = df['возраст'][df['город'] == city].fillna(md[city])
    else:
        df['возраст'][df['город'] == city] = df['возраст'][df['город'] == city].fillna(df['возраст'].median())

In [35]:
gb = df.groupby('город')['пол']
rel = gb.apply(lambda x: x.value_counts()['Мужской'] / x.value_counts()['Женский'] if x.value_counts().index.size else 0).sum() / gb.apply(lambda x: x.isna().sum() == 0).sum()
for key, it in gb:
    ln = len(it)
    f = int(round(ln / (rel + 1)))
    m = ln - f
    df['пол'].fillna({it.index[i]: ('Мужской' if i < m else 'Женский') for i in range(ln)}, inplace=True)

In [5]:
df["возраст"] = df['возраст'].astype(int)
df.loc[df['рост'] <= df['рост'].quantile(0.01), 'рост'] = np.nan
gb1 = df.groupby(['возраст'])['рост']
for key, item in gb1:
    key_mean = gb1.get_group(key).median()
    df["рост"] = df["рост"].fillna(key_mean)

In [35]:
gb2 = df.groupby(['возраст'])['вес']
for key, item in gb2:
    key_mean = gb2.get_group(key).median()
    df["вес"] = df["вес"].fillna(key_mean)

In [7]:
gb = df.groupby(['способ_обогрева', 'способ_охлаждения', 'режим_при_смешанном_типе_охлаждения', 'утепление'])['ощущение_движения_воздуха_(bool)']

In [40]:
gb = df.groupby(['способ_обогрева', 'способ_охлаждения', 'режим_при_смешанном_типе_охлаждения'])['ощущение_движения_воздуха_(bool)']
rel = df['ощущение_движения_воздуха_(bool)'].value_counts()[0] / df['ощущение_движения_воздуха_(bool)'].value_counts()[1]
for key, it in gb:
    ln = len(it)
    o = int(round(ln / (rel + 1)))
    z = ln - o
    df['ощущение_движения_воздуха_(bool)'].fillna({it.index[i]: (1 if i < o else 0) for i in range(ln)}, inplace=True)

In [44]:
df.to_csv('corrected_data.csv')